In [14]:
import json
import pandas as pd
import os

In [15]:
class Json2CSVConverter:
    def __init__(self, csv_paths_list, output_file_path):
        self.csv_paths_list = csv_paths_list
        self.output_file_path = output_file_path
        self.df_list = []
        self.final_df = None
        
    def read_json(self, filename: str) -> dict:
        try:
            with open(filename, "r") as f:
                data = json.loads(f.read())
        except:
            raise Exception(f"Reading {filename} file encountered an error")
        return data

    def normalize_json(self, data: list) -> list:
        new_data=[]
        for single_json in data:
            new_single_json = dict()
            for key, value in single_json.items():
                if not isinstance(value, dict):
                    new_single_json[key] = value
                else:
                    for k, v in value.items():
                        new_single_json[key + "_" + k] = v
            new_data.append(new_single_json)
        return new_data
    
    def get_df(self):
        return self.final_df
    
    def convert(self):
        self.final_df = pd.DataFrame()
        for f in self.csv_paths_list:
            data = self.read_json(filename=f)
            new_data = self.normalize_json(data=data)
            df_new = pd.DataFrame(new_data)
            self.df_list.append(df_new)
            self.final_df = pd.concat([self.final_df, df_new], axis=0)
    
    def save(self, *args, **kwargs):
        self.final_df.to_csv(self.output_file_path, *args, **kwargs)

In [19]:
json_files = [file for file in os.listdir('.') if file.endswith('.json')]
print(json_files)
for file in json_files:
    converter = Json2CSVConverter([file], '{}.csv'.format(file[:-5]))
    converter.convert()
    converter.save(index=False)

['avax_1.json', 'bsc_1.json', 'celo_1.json', 'cro_1.json', 'eth_1.json', 'ftm_1.json', 'ht_1.json', 'movr_1.json', 'one_1.json', 'poly_1.json']


In [54]:
df = converter.get_df()
df.sort_values('timestamp', ignore_index=True, inplace=True)
df.head()

,gasPrice_open,gasPrice_close,gasPrice_low,gasPrice_high,tokenPrice_open,tokenPrice_close,tokenPrice_low,tokenPrice_high,txFee_open,txFee_close,txFee_low,txFee_high,avgGas,timestamp,samples
0,3.40,20.0,0.2,23.0,174.354496,174.354496,174.354496,174.354496,0.018072,0.206292,0.003427,0.206292,52206.937656,2019-10-22T06:39:30.000Z,27
1,1.00,14.0,1.0,20.0,174.354496,174.354496,174.354496,174.354496,0.008803,0.239759,0.007916,0.343440,64993.250965,2019-10-22T14:56:43.000Z,55
2,15.00,2.0,0.5,25.0,174.354496,174.354496,174.354496,174.354496,0.145998,0.027083,0.010553,0.343348,109981.274149,2019-10-22T23:57:03.000Z,61
3,10.00,3.0,1.0,22.0,171.426547,171.426547,171.426547,171.426547,0.081621,0.024352,0.023815,0.145467,133180.156464,2019-10-23T01:35:34.000Z,11
4,2.27,12.0,1.0,20.0,171.426547,171.426547,171.426547,171.426547,0.028528,0.070983,0.005287,0.196194,60807.585902,2019-10-23T09:58:15.000Z,56
